In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import mediapipe as mp

# Load model and labels
model = load_model("hand_sign_model_limited.h5")
labels = np.load("labels_limited.npy")

IMG_SIZE = 64

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get bounding box from landmarks
            x_list = [lm.x * w for lm in hand_landmarks.landmark]
            y_list = [lm.y * h for lm in hand_landmarks.landmark]
            x_min, x_max = int(min(x_list)), int(max(x_list))
            y_min, y_max = int(min(y_list)), int(max(y_list))

            # Add some padding
            padding = 20
            x_min = max(0, x_min - padding)
            y_min = max(0, y_min - padding)
            x_max = min(w, x_max + padding)
            y_max = min(h, y_max + padding)

            # Extract hand ROI
            roi = frame[y_min:y_max, x_min:x_max]
            if roi.size == 0:
                continue

            roi_resized = cv2.resize(roi, (IMG_SIZE, IMG_SIZE))
            roi_normalized = roi_resized.astype('float32') / 255.0
            roi_reshaped = np.expand_dims(roi_normalized, axis=0)

            # Predict
            pred = model.predict(roi_reshaped, verbose=0)
            class_id = np.argmax(pred)
            predicted_label = labels[class_id]

            # Show prediction
            cv2.putText(frame, f'{predicted_label}', (x_min, y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    cv2.imshow("Hand Sign Detection (MediaPipe)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
